In [181]:
import vrep 
import sys
import time 
import numpy as np
from tank import *

import skfuzzy
from skfuzzy import control as ctrl

In [182]:
def parallel_zero_stage(distance_EN_input, distance_ES_input, distance_SE_input, distance_NE_input, tank):

    distance_ES = ctrl.Antecedent(np.arange(0, 6, 0.01), 'distance_ES')
    distance_EN = ctrl.Antecedent(np.arange(0, 6, 0.01), 'distance_EN')

    new_speed = ctrl.Consequent(np.arange(-1, 8, 0.01), 'new_speed')

    distance_EN['poor'] = skfuzzy.trapmf(distance_EN.universe, [0, 0, 1.5, 1.55])
    distance_EN['average'] = skfuzzy.trapmf(distance_EN.universe, [1.5, 1.55, 2.5, 2.51])
    distance_EN['good'] = skfuzzy.trapmf(distance_EN.universe, [2.5, 2.51, 6, 6])

    distance_ES['poor'] = skfuzzy.trapmf(distance_ES.universe, [0, 0, 1.1, 2])
    distance_ES['good'] = skfuzzy.trapmf(distance_ES.universe, [1.1, 2, 6, 6])

    new_speed['zero'] = skfuzzy.trimf(new_speed.universe, [-1, 0, 1])
    new_speed['poor'] = skfuzzy.trimf(new_speed.universe, [0, 3, 5])
    new_speed['good'] = skfuzzy.trimf(new_speed.universe, [5, 6, 7])

    rules = (
        ctrl.Rule(distance_EN['good'] & distance_ES['good'], new_speed['good']),

        ctrl.Rule(distance_EN['poor'] & distance_ES['poor'], new_speed['good']),
        ctrl.Rule(distance_EN['poor'] & distance_ES['good'], new_speed['good']),

        ctrl.Rule(distance_EN['good'] & distance_ES['poor'], new_speed['poor']),

        ctrl.Rule(distance_EN['average'] & distance_ES['good'], new_speed['poor']),

        ctrl.Rule(distance_EN['average'] & distance_ES['poor'], new_speed['zero']),
    )

    driving_ctrl = ctrl.ControlSystem(rules)
    driving = ctrl.ControlSystemSimulation(driving_ctrl)

    driving.input['distance_ES'] = distance_ES_input
    driving.input['distance_EN'] = distance_EN_input

    driving.compute()

    return driving.output['new_speed']

def parallel_first_stage(distance_EN_input, distance_ES_input, distance_SE_input, distance_NE_input, tank):

    distance_ES = ctrl.Antecedent(np.arange(0, 6, 0.01), 'distance_ES')
    distance_SE = ctrl.Antecedent(np.arange(0, 6, 0.01), 'distance_SE')

    new_speed = ctrl.Consequent(np.arange(-1, 8, 0.01), 'new_speed')

    distance_SE['poor'] = skfuzzy.trapmf(distance_SE.universe, [0, 0, 2.4, 3])
    distance_SE['good'] = skfuzzy.trapmf(distance_SE.universe, [2.4, 3, 6, 6])

    distance_ES['poor'] = skfuzzy.trapmf(distance_ES.universe, [0, 0, 1.1, 2])
    distance_ES['good'] = skfuzzy.trapmf(distance_ES.universe, [1.1, 2, 6, 6])

    new_speed['zero'] = skfuzzy.trimf(new_speed.universe, [-1, 0, 1])
    new_speed['poor'] = skfuzzy.trimf(new_speed.universe, [0, 3, 5])
    new_speed['good'] = skfuzzy.trimf(new_speed.universe, [5, 6, 7])

    # dismal poor mediocre average decent good excellent
    rules = (
    ctrl.Rule(distance_SE['good'] & distance_ES['good'], new_speed['good']),
    ctrl.Rule(distance_SE['good'] & distance_ES['poor'], new_speed['zero']),
    ctrl.Rule(distance_SE['poor'] & distance_ES['good'], new_speed['poor']),
    ctrl.Rule(distance_SE['poor'] & distance_ES['poor'], new_speed['good'])
    )

    driving_ctrl = ctrl.ControlSystem(rules)
    driving = ctrl.ControlSystemSimulation(driving_ctrl)

    driving.input['distance_ES'] = distance_ES_input
    driving.input['distance_SE'] = distance_SE_input

    driving.compute()

    return driving.output['new_speed']

def parallel_second_stage(distance_EN_input, distance_ES_input, distance_SE_input, distance_NE_input, tank):

    distance_input = min(distance_ES_input, distance_SE_input)

    distance_ES_SE = ctrl.Antecedent(np.arange(0, 6, 0.01), 'distance_ES_SE')

    new_speed = ctrl.Consequent(np.arange(-1, 8, 0.01), 'new_speed')

    distance_ES_SE['poor'] = skfuzzy.trapmf(distance_ES_SE.universe, [0, 0, 0.79, 0.80])
    distance_ES_SE['good'] = skfuzzy.trapmf(distance_ES_SE.universe, [0.79, 0.80, 6, 6])

    new_speed['zero'] = skfuzzy.trimf(new_speed.universe, [-1, 0, 1])
    new_speed['poor'] = skfuzzy.trimf(new_speed.universe, [0, 1, 2])
    new_speed['good'] = skfuzzy.trimf(new_speed.universe, [5, 6, 7])


    rules = (
            ctrl.Rule(distance_ES_SE['good'], new_speed['good']),
            ctrl.Rule(distance_ES_SE['poor'], new_speed['zero']),
    )

    driving_ctrl = ctrl.ControlSystem(rules)
    driving = ctrl.ControlSystemSimulation(driving_ctrl)
    driving.input['distance_ES_SE'] = distance_input
    driving.compute()

    return driving.output['new_speed']

def parallel_third_stage(distance_SW_input, distance_WS_input, tank):

    distance_input = min(distance_SW_input, distance_WS_input)
    distance_SW_WS = ctrl.Antecedent(np.arange(0, 6, 0.01), 'distance_SW_WS')

    new_speed = ctrl.Consequent(np.arange(-1, 7, 0.01), 'new_speed')

    distance_SW_WS['poor'] = skfuzzy.trapmf(distance_SW_WS.universe, [0, 0, 1.1, 2])
    distance_SW_WS['good'] = skfuzzy.trapmf(distance_SW_WS.universe, [1.1, 2, 6, 6])

    new_speed['zero'] = skfuzzy.trimf(new_speed.universe, [-1, 0, 1])
    new_speed['poor'] = skfuzzy.trimf(new_speed.universe, [0, 1, 2])
    new_speed['good'] = skfuzzy.trimf(new_speed.universe, [5, 6, 7])


    rules = (
            ctrl.Rule(distance_SW_WS['good'], new_speed['good']),
            ctrl.Rule(distance_SW_WS['poor'], new_speed['zero']),
    )

    driving_ctrl = ctrl.ControlSystem(rules)
    driving = ctrl.ControlSystemSimulation(driving_ctrl)
    driving.input['distance_SW_WS'] = distance_input
    driving.compute()

    return driving.output['new_speed']

In [183]:
def perpendicular_zero_stage(distance_NW_input, distance_WN_input, tank):
    distance_WN = ctrl.Antecedent(np.arange(0, 6, 0.01), 'distance_WN')
    distance_NW = ctrl.Antecedent(np.arange(0, 6, 0.01), 'distance_NW')

    new_speed = ctrl.Consequent(np.arange(0, 10, 0.01), 'new_speed')

    distance_WN['poor'] = skfuzzy.trapmf(distance_WN.universe, [0, 0, 1.5, 1.55])
    distance_WN['average'] = skfuzzy.trapmf(distance_WN.universe, [1.5, 1.55, 2.5, 2.51])
    distance_WN['good'] = skfuzzy.trapmf(distance_WN.universe, [2.5, 2.51, 6, 6])

    distance_NW['poor'] = skfuzzy.trapmf(distance_NW.universe, [0, 0, 1.1, 2])
    distance_NW['good'] = skfuzzy.trapmf(distance_NW.universe, [1.1, 2, 6, 6])

    new_speed['zero'] = skfuzzy.trimf(new_speed.universe, [-1, 0, 1])
    new_speed['poor'] = skfuzzy.trimf(new_speed.universe, [0, 3, 5])
    new_speed['good'] = skfuzzy.trimf(new_speed.universe, [5, 6, 7])

    # dismal poor mediocre average decent good excellent
    rules = (
    ctrl.Rule((distance_WN['poor'] | distance_WN['average']) &
              (distance_NW['poor'] | distance_NW['average']), new_speed['poor']),
    ctrl.Rule(distance_WN['good'] & distance_WN['poor'], new_speed['zero'])
    )

    driving_ctrl = ctrl.ControlSystem(rules)
    driving = ctrl.ControlSystemSimulation(driving_ctrl)

    driving.input['distance_NW'] = distance_NW_input
    driving.input['distance_WN'] = distance_WN_input
    driving.compute()

    return driving.output['new_speed']

In [184]:
vrep.simxFinish(-1) # closes all opened connections, in case any prevoius wasnt finished
clientID=vrep.simxStart('127.0.0.1',19999,True,True,5000,5) # start a connection

if clientID!=-1:
    print ("Connected to remote API server")
else:
    print("Not connected to remote API server")
    sys.exit("Could not connect")

#create instance of Tank
tank=Tank(clientID)

Connected to remote API server


In [185]:
proximity_sensors=["EN","ES","NE","NW","SE","SW","WN","WS"]
proximity_sensors_handles=[0]*8

# get handle to proximity sensors
for i in range(len(proximity_sensors)):
    err_code,proximity_sensors_handles[i] = vrep.simxGetObjectHandle(clientID,"Proximity_sensor_"+proximity_sensors[i], vrep.simx_opmode_blocking)
    
#read and print values from proximity sensors
#first reading should be done with simx_opmode_streaming, further with simx_opmode_buffer parameter
for sensor_name, sensor_handle in zip(proximity_sensors,proximity_sensors_handles):
        err_code,detectionState,detectedPoint,detectedObjectHandle,detectedSurfaceNormalVector=vrep.simxReadProximitySensor(clientID,sensor_handle,vrep.simx_opmode_streaming)

In [186]:
from time import sleep

#continue reading and printing values from proximity sensors
distances = dict()
detection_states = dict()

for sensor_name, sensor_handle in zip(proximity_sensors,proximity_sensors_handles):
        err_code,detectionState,detectedPoint,detectedObjectHandle,detectedSurfaceNormalVector=vrep.simxReadProximitySensor(clientID,sensor_handle,vrep.simx_opmode_buffer)
        distances[sensor_name] = np.linalg.norm(detectedPoint)
        detection_states[sensor_name] = detectionState
current_stage = 0
t = time.time()
while (time.time()-t)<100: # read values for 5 seconds

    for sensor_name, sensor_handle in zip(proximity_sensors,proximity_sensors_handles):
        err_code,detectionState,detectedPoint,detectedObjectHandle,detectedSurfaceNormalVector=vrep.simxReadProximitySensor(clientID,sensor_handle,vrep.simx_opmode_buffer)
        distances[sensor_name] = np.linalg.norm(detectedPoint)
        detection_states[sensor_name] = detectionState

    # for i in distances.keys():
    #     if distances[i] > 4:
    #         distances[i] = 4

    if current_stage == 0:
        new_speed = parallel_zero_stage(distances['EN'], distances['ES'], distances['SE'], distances['NE'], tank)
        if new_speed < 3:
            tank.forward(0)
            current_stage += 1
        else:
            tank.forward(new_speed)

    if current_stage == 1:
        new_speed = parallel_first_stage(distances['EN'], distances['ES'], distances['SE'], distances['NE'], tank)
        if new_speed < 1:
            tank.forward(0)
            current_stage += 1
        else:
            tank.forward(new_speed)

    if current_stage == 2:
        new_speed = parallel_second_stage(distances['EN'], distances['ES'], distances['SE'], distances['NE'], tank)
        if new_speed < 1:
            tank.forward(0)
            current_stage += 1
        else:
            tank.leftvelocity = -new_speed
            tank.rightvelocity = -new_speed / 6
            tank.setVelocity()

    if current_stage == 3:
        new_speed = parallel_third_stage(distances['SW'], distances['WS'], tank)
        if new_speed < 1:
            tank.forward(0)
            break
        else:
            tank.leftvelocity = -new_speed / (8/3)
            tank.rightvelocity = -new_speed
            tank.setVelocity()


    print(f"\r{new_speed, tank.readVelocity(), distances['SE'], distances['ES'], distances['EN'], current_stage}", end='')




(1.0136369892000536, -0.6968754300750368, 1.213342445582881, 1.4525081575789362e+23, 1.4525081575789362e+23, 3))